## 2. Vanilla GPFA

It is vanilla in the sense that
#### 1. it only handles trials with the same length for now

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import quantities as pq
from sklearn.decomposition import FactorAnalysis
import matplotlib.pyplot as plt

In [3]:
from e_step import e_step

In [4]:
seqs_train_cut = np.load('simulated_data1.npy',allow_pickle=True)

In [23]:
# ==================================
# Initialize state model parameters
# ==================================
x_dim = 2
bin_width=20.0    # in ms, this should match how we simulated the synthetic data.
tau_init=100.0
eps_init=1.0E-3
    
params_init = dict()
params_init['covType'] = 'rbf' # so far only rbf is implemented for this vanilla version
# GP timescale
# Assume binWidth is the time step size.
params_init['gamma'] = (bin_width / tau_init) ** 2 * np.ones(x_dim)
# GP noise variance
params_init['eps'] = eps_init * np.ones(x_dim)

# ========================================
# Initialize observation model parameters
# ========================================
print('Initializing parameters using factor analysis...')

y_all = np.hstack(seqs_train_cut['y'])
fa = FactorAnalysis(n_components=x_dim, copy=True,
                    noise_variance_init=np.diag(np.cov(y_all, bias=True)))
fa.fit(y_all.T)
params_init['d'] = y_all.mean(axis=1)
params_init['C'] = fa.components_.T
params_init['R'] = np.diag(fa.noise_variance_)

# =====================
# Fit model parameters
# =====================
seqs, ll = e_step(seqs_train_cut, params_init)


Initializing parameters using factor analysis...


In [22]:
#%debug